In [1]:
import pandas as pd
import os

In [5]:
pathogens = ["Acinetobacter baumannii", "Candida albicans", "Campylobacter", "Escherichia coli", "Enterococcus faecium", "Enterobacter",
             "Helicobacter pylori", "Klebsiella pneumoniae", "Mycobacterium tuberculosis", "Neisseria gonorrhoeae", "Pseudomonas aeruginosa",
             "Plasmodium falciparum", "Staphylococcus aureus", "Schistosoma mansoni", "Streptococcus pneumoniae"]

pathogens = ["Mycobacterium tuberculosis"]
root = "."

def get_pathogen_code(pathogen):
    return str(pathogen.split()[0][0] + pathogen.split()[1]).lower() if len(pathogen.split()) > 1 else pathogen.lower()

In [11]:
# For each pathogen
for pathogen in pathogens:

    # Get pathogen code
    pathogen_code = get_pathogen_code(pathogen)

    # Get assay info
    ASSAYS_INFO = pd.read_csv(os.path.join(root, "..", "output", pathogen_code, 'assays.csv'))

    # Load ChEMBL bioactivity data for that pathogen
    print(f"Loading ChEMBL preprocessed data for {pathogen_code}...")
    ChEMBL = pd.read_csv(os.path.join(root, "..", "output", pathogen_code, f"{pathogen_code}_ChEMBL_data.csv"), low_memory=False)
    print(f"Number of activities for {pathogen_code}: {len(ChEMBL)}")
    print(f"Number of compounds for {pathogen_code}: {len(set(ChEMBL['compound_chembl_id']))}")

    # For each assay
    for assay_id, activity_type, unit in zip(ASSAYS_INFO['assay_id'], ASSAYS_INFO['activity_type'], ASSAYS_INFO['unit']):

        print(assay_id, activity_type, unit)

        # Getting ChEMBL bioactivities, compounds and clusters
        if type(unit) == str:
            assay_activities = ChEMBL[(ChEMBL['assay_chembl_id'] == assay_id) & (ChEMBL['activity_type'] == activity_type) & (ChEMBL['unit'] == unit)]["value"].astype(float).tolist()
        else:
            assay_activities = ChEMBL[(ChEMBL['assay_chembl_id'] == assay_id) & (ChEMBL['activity_type'] == activity_type) & (ChEMBL['unit'].isna())]["value"].astype(float).tolist()
        break

Loading ChEMBL preprocessed data for mtuberculosis...
Number of activities for mtuberculosis: 714221
Number of compounds for mtuberculosis: 132378
CHEMBL4649948 PERCENTEFFECT %


In [12]:
assay_activities

[-33.18,
 -3.704,
 -28.81,
 -15.04,
 -9.754,
 -18.26,
 -12.57,
 -13.36,
 -3.278,
 -8.114,
 -19.11,
 0.547,
 -30.85,
 -13.3,
 -7.288,
 -2.38,
 20.88,
 13.21,
 0.6953,
 23.92,
 -28.72,
 -7.949,
 7.151,
 1.49,
 1.956,
 -4.479,
 -18.68,
 1.514,
 7.483,
 -2.456,
 2.498,
 0.3238,
 -7.685,
 -13.26,
 -23.32,
 -5.446,
 -3.239,
 -12.4,
 -6.282,
 14.71,
 -0.9869,
 31.45,
 5.439,
 15.15,
 -12.15,
 -17.47,
 -26.8,
 -7.729,
 -17.95,
 -6.226,
 -12.97,
 -14.34,
 -20.33,
 -14.61,
 -2.294,
 -4.408,
 -19.59,
 -5.378,
 -8.189,
 2.455,
 -23.63,
 -7.529,
 -17.67,
 -4.743,
 -0.3348,
 4.127,
 -9.268,
 3.136,
 26.62,
 0.5545,
 -3.461,
 -1.344,
 -2.217,
 -10.57,
 -6.803,
 -8.387,
 3.781,
 2.14,
 -9.944,
 -2.152,
 9.222,
 16.67,
 5.77,
 15.33,
 -32.28,
 -4.822,
 -14.44,
 -8.753,
 -8.618,
 4.396,
 -9.049,
 8.024,
 1.767,
 -10.87,
 -0.6323,
 -3.837,
 -4.408,
 37.07,
 -1.57,
 -0.4803,
 -3.142,
 10.52,
 -10.21,
 13.09,
 7.104,
 -3.409,
 2.916,
 11.31,
 22.98,
 15.83,
 8.416,
 11.77,
 -4.756,
 10.81,
 10.26,
 4.269,
